In [33]:
import os
from typing import List, Literal, TypedDict
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langgraph.graph import START, END, StateGraph
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [46]:
# ==========================================
# Configuration
# ==========================================
PDF_PATH = "xx.pdf"
CHROMA_PERSIST_DIR = "./chroma_db"
#COLLECTION_NAME = "dspy_book_collection"
COLLECTION_NAME = "rag_test_collection"

In [47]:
# Initialize Global Models
llm_engine = ChatOpenAI(model="gpt-4o-mini")
embedding_engine = OpenAIEmbeddings()

lm_judge = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
)

In [48]:
vector_store = Chroma(
    collection_name=COLLECTION_NAME,  # Must match the original collection name
    embedding_function = embedding_engine,
    persist_directory = CHROMA_PERSIST_DIR   # The directory where data was saved
)

# Now you can run searches immediately without re-adding documents
# results = vector_store.similarity_search("What is the open source model?")

In [49]:
vector_store._collection.count()

254

In [32]:
vector_store.

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': []}

In [45]:
import chromadb
from langchain_community.vectorstores import Chroma

# 1. Setup the native client (We know this works)
client = chromadb.PersistentClient(path=CHROMA_PERSIST_DIR)

# 2. Initialize LangChain using that SAME client
vector_store = Chroma(
    client=client,
    collection_name = "rag_test_collection", # <--- Pass the working client here
    embedding_function=embedding_engine,
)

# 3. Verify connection immediately
print(f"Docs found: {vector_store._collection.count()}")

# 4. Now search
# results = vector_store.similarity_search("What is the open source model?")

Docs found: 254


In [38]:
import chromadb

client = chromadb.PersistentClient(CHROMA_PERSIST_DIR)  # Use your actual path

# 2. Get the collection
collection = client.get_collection(COLLECTION_NAME)
data = collection.get(limit=10)

In [39]:
client.list_collections()

[Collection(name=dspy_book_collection), Collection(name=rag_test_collection)]

In [40]:
client.get_collection('rag_test_collection').get(limit=10)

{'ids': ['fe3eaccb-3de1-4b42-91ef-389f4e1eb033',
  '0e62285f-e6cd-4fcc-bd07-655922770ae5',
  '86fddfa8-1186-4027-99d6-76c03265c3d3',
  'f36d0f97-4ee6-42f2-8fd9-9a6da59d6677',
  'd31a80f3-edf2-4edf-af53-bb5cd005e4a6',
  '1ad76140-f76c-4d82-98cc-8865afbfe6cb',
  '1bba0cfe-f930-45d1-b020-c3a20d5083bf',
  'aa852c7c-0747-4425-b849-a344f6701f95',
  'a8c6d9c5-c883-49f5-8c5a-7a7531008631',
  '255c6714-78a5-4aac-b46a-bfbd7a254550'],
 'embeddings': None,
 'documents': ['',
  'Table of Contents\nChapter 1: DSPy - From Prompting to Programming . . . . . . . . . . . . . . \xa011\nPrompting . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa011\nWhat Makes a Prompt? . . . . . . . . . . . . . . . . . . . . . . . . . \xa011\nPrompt Breakdown . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa011\nDSPy Signature . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa013\nSetting Up Your Environment . . . . . . . . . . . . . . . . . . . . . . . \xa014\nVirtual Environment and P

In [ ]:
clie

In [10]:
# Now you can run searches immediately without re-adding documents
results = vector_store.similarity_search("DSPY")

In [9]:
results

[]

In [ ]:


# 1. Setup & Config


# Using the same model setup as your notebook
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings_fn = OpenAIEmbeddings(model="text-embedding-3-large")

# 2. Vector Store Setup (Reusing your notebook logic)
# Note: Ensure the PDF file exists in your directory
file_path = "compact-guide-to-large-language-models.pdf"
if os.path.exists(file_path):
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    all_splits = text_splitter.split_documents(docs)

    vector_store = Chroma.from_documents(
        documents=all_splits,
        embedding=embeddings_fn,
        collection_name="udacity_refactored"
    )
else:
    print(f"Warning: {file_path} not found. Vector store will be empty.")
    vector_store = Chroma(embedding_function=embeddings_fn, collection_name="empty_test")

# 3. State Definition
class State(TypedDict):
    question: str
    documents: List[Document]
    loop_step: int
    evaluation: str  # "Sufficient" or "Insufficient"
    answer: str

# 4. Nodes

def rewrite_query(state: State):
    """
    First step: Dummy placeholder.
    Subsequent steps: Rewrites the query if retrieval failed.
    """
    question = state["question"]
    loop_step = state.get("loop_step", 0)

    # If it's the very first step, act as dummy/pass-through
    if loop_step == 0:
        print(f"---STEP {loop_step}: INITIAL QUERY PASS-THROUGH---")
        return {"loop_step": loop_step + 1}

    # If looping back, actually rewrite the query
    print(f"---STEP {loop_step}: REWRITING QUERY---")

    system = "You are a helpful assistant that optimizes queries for vector retrieval."
    human = f"Look at the initial question: {question}. Formulate an improved question to find better results."

    msg = [SystemMessage(content=system), HumanMessage(content=human)]
    better_question = llm.invoke(msg).content

    return {"question": better_question, "loop_step": loop_step + 1}

def retrieve(state: State):
    """
    Retrieve documents based on the current (potentially rewritten) question.
    """
    print("---RETRIEVING DOCUMENTS---")
    question = state["question"]
    retrieved_docs = vector_store.similarity_search(question)
    return {"documents": retrieved_docs}

def evaluator(state: State):
    """
    Evaluates if the retrieved documents are sufficient to answer the question.
    """
    print("---EVALUATING DOCUMENTS---")
    question = state["question"]
    documents = state["documents"]
    docs_content = "\n\n".join(doc.page_content for doc in documents)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert evaluator. Given the context, determine if it is sufficient to answer the user question."),
        ("human", "Question: {question}\n\nContext: {context}\n\nIs the context sufficient? Return only 'YES' or 'NO'.")
    ])

    chain = prompt | llm | StrOutputParser()
    score = chain.invoke({"question": question, "context": docs_content})

    status = "Sufficient" if "YES" in score.upper() else "Insufficient"
    print(f"---EVALUATION: {status}---")
    return {"evaluation": status}

def generate(state: State):
    """
    Generates the final answer.
    """
    print("---GENERATING ANSWER---")
    question = state["question"]
    documents = state["documents"]
    docs_content = "\n\n".join(doc.page_content for doc in documents)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant. Use the context to answer the question. If you don't know, say so."),
        ("human", "Question: {question}\n\nContext: {context}\n\nAnswer:")
    ])

    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"question": question, "context": docs_content})
    return {"answer": answer}

# 5. Conditional Logic (Router)

def router(state: State) -> Literal["generate", "rewrite_query"]:
    evaluation = state["evaluation"]
    loop_step = state["loop_step"]

    # If documents are good, generate answer
    if evaluation == "Sufficient":
        return "generate"

    # If documents are bad, check loop limit (max 3 retries)
    # loop_step starts at 1 after first pass.
    # If loop_step > 3, we stop trying and just generate best effort.
    if loop_step <= 3:
        return "rewrite_query"

    return "generate"

# 6. Graph Construction

workflow = StateGraph(State)

# Add Nodes
workflow.add_node("rewrite_query", rewrite_query)
workflow.add_node("retrieve", retrieve)
workflow.add_node("evaluator", evaluator)
workflow.add_node("generate", generate)

# Add Edges
workflow.add_edge(START, "rewrite_query")
workflow.add_edge("rewrite_query", "retrieve")
workflow.add_edge("retrieve", "evaluator")

# Add Conditional Edge from Evaluator
workflow.add_conditional_edges(
    "evaluator",
    router,
    {
        "rewrite_query": "rewrite_query",
        "generate": "generate"
    }
)

workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

# 7. Execution (Example)
from IPython.display import Image, display

# Visualize
try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    pass

# Run
initial_input = {"question": "What is the open source model?", "loop_step": 0}
result = app.invoke(initial_input)

print("\nFinal Answer:")
print(result["answer"])